In [1]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input, decode_predictions

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array, array_to_img

from keras import Model
from keras.layers import Dense, Flatten
from keras.optimizers import SGD
from keras.utils import to_categorical
import pandas as pd

from random import randint

Using TensorFlow backend.


#### Nous avons utilisé le modèle inceptionV3 au lieu de VGG16 afin d'utilisé un autre modèle autre que celui utilisé en classe, selon nos recherches ce modèles admet un nombre de couches plus grand que celui de VGG16 et la taille d'input par défaut est égale à 299x299 contrairement au VGG16 qui est 224x224.**

In [2]:
incep3 = InceptionV3()

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
def predict_image_using_inceptionv3(img_path, vgg):
    """
    Open image, convert it to an array, and run predictions
    using a trained model.
    """
    # load an image from file
    image = load_img(img_path, target_size=(299, 299))
    # convert the image pixels to a numpy array
    image = img_to_array(image)
    # reshape data for the model
    image = image.reshape(
        (1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess_input(image)
    y_pred = vgg.predict(image)
    labels = decode_predictions(y_pred)[0]
    for label in labels:
        print(label)
        

### Creating a dataset

#### Ici on a rajouté un test "label != '.DS_Store'" parceque il y a un dossier caché à chaque fois il est créé automatiquement et on a pas réussi à le supprimer 

In [ ]:
import os
import glob
import numpy as np

def create_dataset(folder):
    all_images = []
    image_arrays = []
    all_labels = []
    filenames = []
    labels = []
    for class_id, label in enumerate(os.listdir(folder)):
        if (label != '.DS_Store'):
            print(class_id-1, label)
            labels.append(label)
            for filename in glob.glob(folder + '/{}/*'.format(label)):
                filenames.append(filename)
                im = load_img(filename, target_size=(299, 299))
                all_images.append(im)
                preprocessed = preprocess_input(img_to_array(im))
                image_arrays.append(preprocessed)
                all_labels.append(class_id-1)
    X = np.array(image_arrays)
    Y = to_categorical(np.array(all_labels))
    return X, Y, filenames, all_images, labels

#### Quelques affiches pour notre dataset

In [ ]:
X, Y, filenames, all_images, labels = create_dataset('../Datas/img/Fashion_train')

In [ ]:
X.shape, Y.shape, len(filenames), len(all_images)

In [ ]:
Y

In [ ]:
filenames[:8]

**Prédiction sur une image choisie aléatoirement depuis notre Dataset avec le modèle CNN inceptionV3**

In [ ]:
img_id = randint(0,len(X))
predict_image_using_inceptionv3(filenames[img_id], incep3)
all_images[img_id]

### Transfer learning

In [ ]:
# Charger InceptionV3 pré-entraîné sur ImageNet
# mais sans les couches fully-connected
inceptionv3_cut = InceptionV3(weights="imagenet", include_top=False,
                input_shape=(299, 299, 3))
x = inceptionv3_cut.output
# transform matrix into 1-d vector
x = Flatten()(x)

# Layer  
x = Dense(40, activation='relu')(x) 

# Nombre de classe
nb_classes = 7
x = Dense(nb_classes, activation='softmax')(x)  



In [ ]:
custom_model = Model(inputs=inceptionv3_cut.input, outputs=x)

In [ ]:
for layer in custom_model.layers[:-2]:
    layer.trainable = False

In [ ]:
custom_model.summary()

In [ ]:
# Compiler le modèle 
custom_model.compile(
    loss="categorical_crossentropy",
    optimizer=SGD(lr=0.0001, momentum=0.9),
    metrics=["accuracy"])

In [ ]:
# Entraîner le modèle
custom_model.fit(X, Y,
                 epochs=10, batch_size=5)

#### Après avoir tester plusieurs epoch de 3 à 10, les résultats s'améliorent avec un nombre de perte minimale et un score élevé

#### Utilisons notre modèle entrainé pour faire des prédictions sur les images de test

In [ ]:
# télécharger les images de test
def load_test_set(folder):
    all_images = []
    image_arrays = []
    filenames = []
    for filename in glob.glob(folder + '/*'):
        filenames.append(filename)
        im = load_img(filename, target_size=(299, 299))
        all_images.append(im)
        preprocessed = preprocess_input(img_to_array(im))
        image_arrays.append(preprocessed)
    X = np.array(image_arrays)
    return X, filenames, all_images

In [ ]:
X_test, test_filenames, test_images = load_test_set(
    '../Datas/img/Fashion_Test')

In [ ]:
X_test.shape

#### Nous avons utilisé randint pour choisir aléatoirement les images pour le test

In [ ]:
test_id = randint(0,len(X_test))
test_images[test_id]

In [ ]:
img_reshaped = X_test[test_id].reshape(1, *X_test[test_id].shape)
pred = custom_model.predict(img_reshaped)

In [ ]:
pd.DataFrame(pred*100, columns=labels)

In [ ]:
test_id1 = randint(0,len(X_test))
test_images[test_id1]

In [ ]:
img_reshaped1 = X_test[test_id1].reshape(1, *X_test[test_id1].shape)
pred1 = custom_model.predict(img_reshaped1)

In [ ]:
pd.DataFrame(pred1*100, columns=labels)

In [ ]:
test_id2 = randint(0,len(X_test))
test_images[test_id2]

In [ ]:
img_reshaped2 = X_test[test_id2].reshape(1, *X_test[test_id2].shape)
pred2 = custom_model.predict(img_reshaped2)

In [ ]:
# Prediction de catégorie
pd.DataFrame(pred2*100, columns=labels)

#### **** Après avoir tester plusieurs images de différentes catégories, notre modèle costum a réussi à prédire toutes les images avec une poucentage supèrieur à 92%  ***
